In [1]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz
!gunzip /content/cc.fa.300.bin.gz
!wget https://raw.githubusercontent.com/Mehrdadghassabi/Persian-Sentiment-Analyzer/master/better_dataset/test.csv
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=12CrgrdQF7X91s2fOF7ubjQOFWRVEjlkD' -O 'best_model.h5'
!pip install fasttext
!pip install hazm

--2022-02-20 16:28:44--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4502524724 (4.2G) [application/octet-stream]
Saving to: ‘cc.fa.300.bin.gz’

cc.fa.300.bin.gz    100%[===================>]   4.19G  32.9MB/s    in 2m 14s  

2022-02-20 16:30:59 (32.1 MB/s) - ‘cc.fa.300.bin.gz’ saved [4502524724/4502524724]

--2022-02-20 16:32:18--  https://raw.githubusercontent.com/Mehrdadghassabi/Persian-Sentiment-Analyzer/master/better_dataset/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [2]:
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import load_model
import fasttext 
import pandas
import hazm
import numpy as np
import keras.backend as K

def clean_text(text):
  normalizer = hazm.Normalizer()
  text = normalizer.normalize(text)
  return text

the_size = 300 
maxlen = 20
model = load_model('best_model.h5')
Word_vectors = fasttext.load_model("/content/cc.fa.300.bin")

In [3]:
test = pandas.read_csv("/content/test.csv", sep='	')
test_list = list(map(lambda x: [clean_text(x[0]),x[1]],zip(test['comment'],test['label_id'])))
test_size=len(test_list)
x_test = np.zeros((test_size, maxlen, the_size), dtype=K.floatx())
y_test = np.zeros((test_size, 2), dtype=np.int32)

for i in range(test_size):
  text_words = hazm.word_tokenize(test_list[i][0])
  for t in range(0,len(text_words)):
    if t >= maxlen:
      break
    if text_words[t] not in Word_vectors.words:
      continue
    x_test[i, t, :] = Word_vectors.get_word_vector(text_words[t])
  if test_list[i][1] == 1:
    y_test[i, :] = [1.0, 0.0]
  else:
    y_test[i, :] = [0.0, 1.0]

pred = model.predict(x_test)
y_pred = np.where(pred > 0.5, 1, 0)

predod =np.empty(len(y_pred))
testod =np.empty(len(y_test))

for x in range(len(y_pred)) :
    if y_pred[x][0] == 1 :
      predod[x] = 0
    else :
      predod[x] = 1

for y in range(len(y_test)) :
    if y_test[y][0] == 1 :
      testod[y] = 0
    else :
      testod[y] = 1
print(confusion_matrix(y_true=predod, y_pred=testod))
print(classification_report(y_true=predod, y_pred=testod))

[[3052  749]
 [ 448 2751]]
              precision    recall  f1-score   support

         0.0       0.87      0.80      0.84      3801
         1.0       0.79      0.86      0.82      3199

    accuracy                           0.83      7000
   macro avg       0.83      0.83      0.83      7000
weighted avg       0.83      0.83      0.83      7000

